# Equity Linked Notes - Autocallable Phoenix Memory (single)

This notebook demonstrates how to access and use the functionalities of **Autocallable Structured Notes** which are part of our **QPS** module within LSEG Financial Analytics SDK. This notebook presents a Phoenix Memory structure with single underlying.

**You will be able to:**
* Define Phoenix Memory instrument with basic parameters
* Configure pricing parameters
* Evaluate Phoenix Memory Analytics

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import structured_products as sp 

import json
import datetime as dt
import pandas as pd
from IPython.display import display

## Data Preparation

To define a Structured Product instrument, in this example Autocallable Phoenix Memory, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (strike, dates, notional, index)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing parameters, optional

In [2]:
# 1. Create SP definition object

SN_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "Phoenix_Memory",
    inputs = [
        sp.NameTypeValue(name="notional", type = "string", value="1000"),
        sp.NameTypeValue(name="Underlying", type = "string", value="ENEI_MI"),
        sp.NameTypeValue(name="StrikeDate", type = "date", value="04/08/2025"),
        sp.NameTypeValue(name="MaturityDate", type = "date", value="11/08/2031"),
        sp.NameTypeValue(name="LastValuationDate", type = "date", value="04/08/2031"),
        sp.NameTypeValue(name="Schedule", type = "schedule", value=[
                    ["04/08/2026", "04/08/2026", "11/08/2026", "11/08/2026"],
                    ["04/08/2027", "04/08/2027", "11/08/2027", "11/08/2027"],
                    ["04/08/2028", "04/08/2028", "11/08/2028", "11/08/2028"],
                    ["06/08/2029", "06/08/2029", "13/08/2029", "13/08/2029"],
                    ["05/08/2030", "05/08/2030", "12/08/2030", "12/08/2030"]]),
        sp.NameTypeValue(name="CouponBarrier", type = "curve", value=[["04/08/2025","70%"]]),
        sp.NameTypeValue(name="FinalCouponBarrier", type = "string", value="70%"),
        sp.NameTypeValue(name="AutocallBarrier", type = "curve", value=[["04/08/2025","100%"]]),
        sp.NameTypeValue(name="RedemptionBarrier", type = "string", value="50%"),
        sp.NameTypeValue(name="CouponRate", type = "string", value="5.75%"),
        sp.NameTypeValue(name="Leverage", type = "string", value="100%"),
        sp.NameTypeValue(name="VarInit", type = "string", value="0"),
        sp.NameTypeValue(name="NbPeriodNoCallable", type = "string", value="0")
    ],
    payoff_description = [
          [
            "Schedule type",
            "Schedule description",
            "Index",
            "Performance",
            "Alive",
            "Early",
            "Count",
            "Coupon",
            "SumOfCoupons",
            "Settlement",
            "OptionAtMaturity",
            "PriceIn%",
            "Price"
          ],
          [
            "AtDate",
            "StrikeDate",
            "EqSpot(Underlying)",
            "",
            "1",
            "",
            "$n=VarInit;$n",
            "",
            "0",
            "",
            "",
            "",
            ""
          ],
          [
            "OnUserSchedule",
            "Schedule",
            "EqSpot(Underlying)",
            "Index[t] / Index[StrikeDate]",
            "If($n<NbPeriodNocallable,1,If(Performance[t]>=Interpol(AutocallBarrier,PS()),0,Alive[LastDate]))",
            "(1 - Alive[t]) * Alive[LastDate(-1)]",
            "$n=$n+1;$n",
            "Receive If(Performance[t]>=Interpol(CouponBarrier,PS()),$n*CouponRate-SumOfCoupons[LastDate(-1)],0)*Alive[LastDate(-1)]",
            "Coupon[t]+SumOfCoupons[LastDate]",
            "Receive Early[t]",
            "",
            "",
            ""
          ],
          [
            "AtDate",
            "LastValuationDate",
            "EqSpot(Underlying)",
            "Index[t] / Index[StrikeDate]",
            "",
            "",
            "$n=$n+1;$n",
            "Receive (MaturityDate, If(Performance[t]>=FinalCouponBarrier, $n*CouponRate-SumOfCoupons[LastDate], 0) * Alive[LastDate])",
            "",
            "Receive (MaturityDate, Alive[LastDate])",
            "Receive (MaturityDate, If(Performance[t]>=RedemptionBarrier,0,(Performance[t] / Leverage -1)*Alive[LastDate]))",
            "Report((columnval(Coupon)+columnval(OptionAtMaturity)+columnval(Settlement))*100)",
            "Report(columnval(PriceIn%)/100*Notional)"
          ]
        ]
)


# 2. Create SP instrument definition object

phoenix_memory = sp.StructuredProductsDefinitionInstrument(definition = SN_definition)
print("Instrument definition created")


# 3. Create SP parameters object - optional

phoenix_memory_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2025, 10, 15),  # Set your desired valuation date
    report_ccy="EUR",  # Set your reporting currency
    numerical_method = sp.GenericNumericalMethod(method="MonteCarlo"),
    models=[sp.ModelDefinition(
            underlying_code = "ENEI.MI",
            underlying_name = "ENEI_MI",
            underlying_currency = "EUR",
            asset_class = "Equity",
            model_name= "Dupire")]
)

print("Pricing parameters configured")

Instrument definition created
Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing
try:
    response = sp.price(
        definitions=[phoenix_memory],
        pricing_preferences=phoenix_memory_pricing_params,
        market_data=None,
        return_market_data=True,  # or False
        fields=None  # or specify fields as a string
    )

    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing execution completed")
    
except Exception as e:
    print(f"Price calculation failed: {str(e)}")
    raise

Pricing execution completed


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., strike, dates, notional, underlying, barrier, profit target), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method.

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` (payments, fixings)
   - **description**: Instrument summary StructuredProductDefinition and also the default fields not specified in the StructuredProductDefinition, but used by default in the calculation
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

#### Description
Useful for understanding which fields are included by default in the price function, even if they are not explicitly specified.

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "Phoenix_Memory",
    "dealCcy": "EUR",
    "reportCcy": "EUR",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "Alive": 1.426542485,
        "Count": 19.247330964,
        "Coupon": 0.101216723,
        "Early": 0.753242218,
        "Index": 40.562123706,
        "OptionAtMaturity": -0.092501443,
        "Performance": 5.164517915,
        "Price": 961.82360261,
        "PriceIn%": 96.182360261,
        "Settlement": 0.953108323,
        "SumOfCoupons": 0.382373089
    },
    "processingInformation": "'PricePercent' column was not retrieved in payoff description: related fields will not be retrieved. The following greeks could not be retrieved: Theta, Vega, Gamma, Delta. Make sure a 'PricePercent' column is specified in payoff description to compute greeks. "
}


#### Analytics

##### Valuation

In [5]:
# Extract vauation from the response
valuation = response.data.analytics[0].valuation

# Convert the dictionary to a DataFrame
df_phoenix_memory_valuation = pd.DataFrame(list(valuation.items()), columns=["Field", "Value"])

display(df_phoenix_memory_valuation)

,Field,Value
0,marketValueInDealCcy,961.823603
1,marketValueInReportCcy,961.823603


##### Cash Flows

In [6]:
# Extract cashflows from response
cashflows = response.data.analytics[0].cashflows["cashFlows"]

# Extract underlyings
model_df = pd.DataFrame(data=phoenix_memory_pricing_params.models)
underlying_list = model_df['underlyingName'].to_list()

# Build dataframes for all cash flow types
output = []
for cf_type in cashflows:
    cashflow_df = pd.DataFrame(cf_type['payments'])
    if cf_type['legTag'] == 'Index':
        cashflow_df = cashflow_df.rename(columns={'amount': underlying_list[0]})
    else:
        cashflow_df = cashflow_df.rename(columns={'amount': cf_type['legTag']})
    cashflow_df['discountFactor'] = cashflow_df['discountFactor'].round(4)
    output.append(cashflow_df)

# Merge all dataframes on the 'date' column
combined_df = pd.concat([*output],axis=1)

# Remove duplicated columns with the same values
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

common_cols = ['date', 'discountFactor', 'Performance', 'Alive', 'Early', 'Coupon', 'Settlement', 'OptionAtMaturity', 'Price', 'currency', 'occurence']
indv_perf = [f'PERF_{underlying}' for underlying in underlying_list if len(underlying_list) > 1]
cols_to_display = common_cols[0:2] + underlying_list + indv_perf + common_cols[2:]

# Leave only columns to display
combined_df = combined_df.loc[:,[*cols_to_display]]

# Display the combined dataframe
display(combined_df)

,date,discountFactor,ENEI_MI,Performance,Alive,Early,Coupon,Settlement,OptionAtMaturity,Price,currency,occurence
0,2025-08-04,1.0000,7.854000,0.000000,1.0000,0.0000,0.000000,0.000000,0.000000,0.000000,EUR,Historical
1,2026-08-04,0.9843,8.081583,1.028977,0.4216,0.5784,0.055341,0.578189,0.000000,0.000000,EUR,Future
2,2027-08-04,0.9659,7.727828,0.983935,0.3084,0.1132,0.017709,0.113157,0.000000,0.000000,EUR,Future
3,2028-08-04,0.9461,7.384789,0.940258,0.2896,0.0188,0.011074,0.018792,0.000000,0.000000,EUR,Future
4,2029-08-06,0.9249,7.004557,0.891846,0.2532,0.0364,0.009946,0.036384,0.000000,0.000000,EUR,Future
5,2030-08-05,0.9028,6.703076,0.853460,0.2276,0.0256,0.006439,0.025588,0.000000,0.000000,EUR,Future
6,2031-08-04,0.8798,6.394036,0.814112,0.0000,0.0000,0.004714,0.227484,-0.105135,1093.184908,EUR,Future


##### Greeks

In [7]:
# Extract Greeks from the response
greeks = response.data.analytics[0].greeks

# Convert the dictionary to a DataFrame
df_phoenix_memory_greeks = pd.DataFrame(list(greeks.items()), columns=["Greeks", "Value"])

display(df_phoenix_memory_greeks)

,Greeks,Value
0,deltaAmountInDealCcy,5.655000
1,deltaAmountInReportCcy,5.655000
2,gammaAmountInDealCcy,-1.721670
3,gammaAmountInReportCcy,-1.721670
4,vegaAmountInDealCcy,-8.378800
5,vegaAmountInReportCcy,-8.378800
6,thetaAmountInDealCcy,0.318712
7,thetaAmountInReportCcy,0.318712
